In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn import linear_model
import sys

In [3]:
datafile1 = open('house_with_missing.csv', 'r') 
f= pd.read_csv(datafile1)


In [6]:
def brief(f):
    
    file1 = open("file.txt","w")
    
    file1.write("\n-----------------------------------")
    file1.write("\nThis dataset has"+" "+str(f.shape[0])+" "+"Rows"+" "+str(f.shape[1])+" "+"Atributes")
    file1.write("\n\nreal value attributes")
    file1.write("\n-----------------------------------")
    
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    a=f.describe(include=numerics).round(2)
    d=f.describe(include='all').T.reset_index().rename(columns={"index": "Attribute_Name"}).round(2)
    
    b= a.T.reset_index().rename(columns={"index": "Attribute_Name"})
    
    tr = pd.DataFrame({'Attribute_ID': [i for i in range(1,d.shape[0]+1)]})
    tr=tr.join(d.Attribute_Name)
    
    i=np.array(d['Attribute_Name'].reset_index(drop = True))
    ii=np.array(b['Attribute_Name'].reset_index(drop = True))
    fin= np.isin(i, ii)
    final= np.where(fin)

    sublist=[list(i) for i in final]
    flat_list0 = list(np.concatenate(final))
    flat_list= [x+1 for x in flat_list0]  #for numeric
    
    x = {v:k for v,k in enumerate(fin) if k == False}
    d=[]

    for name, age in x.items(): 
        d.append(name)
    
    flat0= [x for x in d] #for object
    flat= [x+1 for x in d] #for object

    df_drop = f.drop(f.columns[flat0], axis=1)  #for numeric
    df_dropo = f.drop(f.columns[flat_list0], axis=1)  #for object
    
    #b.index=b.index + 1
    b= b.rename(columns={"50%":"Median","mean":"Mean","std":"Sdev","min":"Min","max":"Max"})
    
    df = pd.DataFrame({'Attribute_ID': flat_list})
    
    
    df=df.join(b.Attribute_Name)
    df=df.join(pd.DataFrame({'Missing': df_drop.isnull().sum(axis=0).tolist()}))
    df=df.join(b['Mean'])
    df=df.join(b['Median'])
    df=df.join(b['Sdev'])
    df=df.join(b['Min'])
    df=df.join(b['Max'])
    df.index += 1 
    
    file1.write("\n"+str(df.to_string()))
    file1.write("\n\nsymbolic attributes")
    file1.write("\n-----------------------------------")           
    
    a=f.describe(include=object).T.reset_index().rename(columns={"index": "Attribute_Name","unique":"arity"})
    
    af = pd.DataFrame({'Attribute_ID': flat})
    af=af.join(a['Attribute_Name'])   
    af=af.join(pd.DataFrame({'Missing': df_dropo.isnull().sum(axis=0).tolist()}))
    af=af.join(a['arity']) 
    
    
    
    df_dropo.dropna(inplace=True)
    m,n= df_dropo.shape
    df_dropo = df_dropo.astype(str)
    
    
    f = pd.DataFrame(columns=['MCVs_counts'])
    for i in range(n):
        uni,counts= np.unique(df_dropo[df_dropo.columns[i]],return_counts=True)
        c = sorted(counts,reverse=True)
        count_sort_ind = np.argsort(-counts)
        unique_des= uni[count_sort_ind]
        uni=unique_des.tolist()
        co=count_sort_ind.tolist()
        if (len(unique_des))==1:
            f= f.append({'MCVs_counts':str(unique_des[0])+'('+ str(c[0])+')'},ignore_index=True)
        elif (len(unique_des))==2:
            f= f.append({'MCVs_counts':str(unique_des[0])+'('+ str(c[0])+')'+" "+str(unique_des[1])+'('+ str(c[1])+')'},ignore_index=True)
        else:
            f= f.append({'MCVs_counts':str(unique_des[0])+'('+ str(c[0])+')'+" "+str(unique_des[1])+'('+ str(c[1])+')'+str(unique_des[2])+'('+ str(c[2])+')'},ignore_index=True)
    
    #af=af.concat(f['MCVs_counts'])    
    f['MCVs_counts'] = f['MCVs_counts'].astype(str)
    #aff=pd.concat([af, f],axis=0)
    af=af.join(f) 
    af.index += 1 
    
    file1.write("\n"+str(af.to_string()))
        
    file1.close()
    return 

brief(f)